In [3]:
import csv
import pandas as pd

In [27]:
!pip install sklearn

Processing c:\users\kasutaja\appdata\local\pip\cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074\sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.22.1-cp37-cp37m-win_amd64.whl (6.3 MB)


In [4]:
extract = pd.read_csv("egcut_epi_nesma_extracted_features_example.csv",header=None)

In [5]:
extract.columns = ["text_ID","start","end","object","value","unit","min","max"]

extract.head()

,text_ID,start,end,object,value,unit,min,max
0,46809,0,6,RR,130.0,NaN,NaN,NaN
1,45389,165,171,RR,110.0,NaN,NaN,NaN
2,47482,0,5,RR,142.0,NaN,NaN,NaN
3,53393,0,5,RR,143.0,NaN,NaN,NaN
4,53571,0,6,RR,140.0,NaN,NaN,NaN


In [6]:
text = pd.read_csv("egcut_epi_nesma_texts_example.csv",header=None)

In [7]:
text.columns = ["text_ID","text"]
text.head()

,text_ID,text
0,347474,\nRR 132/84 \n\n
1,46809,RR 130/80 mmHg.
2,45389,"Kergelt liigkaaluline. Nahk, limaskestad taval..."
3,47482,"RR142/89mmHg, HR 76 x min, p167cm, k89kg. EKG..."
4,53393,"RR143/87mmHg, HR60 x min, p180cm, k93kg"


In [8]:
text.text_ID.value_counts()

428132     61
440167     31
284004     28
179052     28
1118428    25
           ..
1644002     1
356033      1
539481      1
429757      1
804663      1
Name: text_ID, Length: 121535, dtype: int64

In [9]:
text = text.drop_duplicates(subset='text_ID', keep='first')

In [10]:
len(text)

121535

In [11]:
len(extract)

185888

In [12]:
DS = pd.merge(extract, text, how="left", on='text_ID')

In [13]:
len(DS)

185888

In [14]:
DS.head()

,text_ID,start,end,object,value,unit,min,max,text
0,46809,0,6,RR,130.0,NaN,NaN,NaN,RR 130/80 mmHg.
1,45389,165,171,RR,110.0,NaN,NaN,NaN,"Kergelt liigkaaluline. Nahk, limaskestad taval..."
2,47482,0,5,RR,142.0,NaN,NaN,NaN,"RR142/89mmHg, HR 76 x min, p167cm, k89kg. EKG..."
3,53393,0,5,RR,143.0,NaN,NaN,NaN,"RR143/87mmHg, HR60 x min, p180cm, k93kg"
4,53571,0,6,RR,140.0,NaN,NaN,NaN,RR 140/74 mm Hg Cor fr. 62 UH uuringul kaela a...


In [15]:
DS.isna().sum()

text_ID         0
start           0
end             0
object          0
value           0
unit       166347
min        185137
max        180579
text            0
dtype: int64

In [16]:
DS.object.value_counts()

RR        138495
eGFR       25406
Pikkus     21987
Name: object, dtype: int64

In [17]:
DS["object"].unique()

array(['RR', 'Pikkus', 'eGFR'], dtype=object)

In [18]:
DS.text_ID.value_counts()

428132     61
440167     31
284004     28
179052     28
1118428    25
           ..
345576      1
89579       1
427708      1
99824       1
1221283     1
Name: text_ID, Length: 121535, dtype: int64

In [19]:
DS.head()

,text_ID,start,end,object,value,unit,min,max,text
0,46809,0,6,RR,130.0,NaN,NaN,NaN,RR 130/80 mmHg.
1,45389,165,171,RR,110.0,NaN,NaN,NaN,"Kergelt liigkaaluline. Nahk, limaskestad taval..."
2,47482,0,5,RR,142.0,NaN,NaN,NaN,"RR142/89mmHg, HR 76 x min, p167cm, k89kg. EKG..."
3,53393,0,5,RR,143.0,NaN,NaN,NaN,"RR143/87mmHg, HR60 x min, p180cm, k93kg"
4,53571,0,6,RR,140.0,NaN,NaN,NaN,RR 140/74 mm Hg Cor fr. 62 UH uuringul kaela a...


In [20]:
# X = DS.loc[:, DS.columns != 'object']
X = DS.text
Y = DS.object

In [28]:
import numpy as np
from keras.utils import np_utils

import matplotlib.pyplot as plt
from keras.layers import GRU,Dropout,BatchNormalization
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

%matplotlib inline

In [29]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [30]:
shape = 3
encoder = LabelEncoder()
encoder.fit(Y)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=1,stratify = Y)

Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size=0.25, random_state=1,stratify = Ytrain)



In [31]:
len(Xtrain)

111532

In [32]:
Ytrain = encoder.transform(Ytrain)
Ytest = encoder.transform(Ytest)
Yval = encoder.transform(Yval)

Ytrain = np_utils.to_categorical(Ytrain)
Ytest = np_utils.to_categorical(Ytest)
Yval = np_utils.to_categorical(Yval)



In [33]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(Xtrain)

Xtrain = tokenizer.texts_to_sequences(Xtrain)
Xtest = tokenizer.texts_to_sequences(Xtest)
Xval = tokenizer.texts_to_sequences(Xval)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index



In [34]:
maxlen = 1000

Xtrain = pad_sequences(Xtrain, padding='post', maxlen=maxlen)
Xtest = pad_sequences(Xtest, padding='post', maxlen=maxlen)
Xval = pad_sequences(Xval, padding='post', maxlen=maxlen)



In [35]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(GRU(256))
model.add(Dropout(0.1))
model.add(layers.Dense(shape, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 50)          15273550  
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 15,510,097
Trainable params: 15,510,097
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(Xtrain, Ytrain,
                    epochs=10,
                    verbose=True,
                    validation_data=(Xval, Yval),batch_size=1000)


C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 111532 samples, validate on 37178 samples
Epoch 1/10
 25000/111532 [=====>........................] - ETA: 4:32:46 - loss: 0.8411 - accuracy: 0.7221

In [ ]:

f= open("report.csv","a")
loss, accuracy = model.evaluate(Xtrain, Ytrain, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
f.write(",{:.4f}".format(accuracy))

loss, accuracy = model.evaluate(Xval, Yval, verbose=True)

print("Validation Accuracy:  {:.4f}".format(accuracy))
f.write(",{:.4f} \n".format(accuracy))
f.close()
plot_history(history)
Ypred = model.predict(Xtest)
from sklearn.metrics import confusion_matrix, classification_report

matrix = confusion_matrix(Ytest.argmax(axis=1), Ypred.argmax(axis=1))
classification_Report = classification_report(Ytest.argmax(axis=1), Ypred.argmax(axis=1), output_dict=True)
df = pd.DataFrame(classification_Report).transpose()
df.to_csv("ClassificationReport.csv")

In [82]:
full_extract = pd.read_csv("/home/nesma/egcut_epi_nesma_extracted_features.csv",header=None)

/home/nesma/anaconda3/envs/basic/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
full_extract.head()

,0,1,2,3,4,5,6,7
0,46809,0,6,RR,130,NaN,NaN,NaN
1,43588,2421,2433,ALP,54,U/L,NaN,NaN
2,43050,720,735,HCT,40,%,NaN,41.2
3,45389,165,171,RR,110,NaN,NaN,NaN
4,42960,2314,2332,HCT,45,%,40.0,50.0


In [85]:
full_extract.columns  = ["id","start","end","object","value","unit","min","max"]

In [93]:
full_extract["id"].value_counts().head(n=20)

428132     1172
440167      472
284004      397
179052      397
121117      350
450576      310
737895      271
999932      265
201723      233
1111922     233
438105      229
200487      226
966450      204
238631      203
213615      199
419013      195
288904      191
211700      190
325959      190
85772       189
Name: id, dtype: int64

In [95]:
 full_extract[full_extract["id"] == 85772]

,id,start,end,object,value,unit,min,max
94143,85772,813,841,Lümfotsüüdid,7.5,%,20.0,40.0
94144,85772,845,878,Neutrofiilid,86.5,%,40.0,75.0
94145,85772,897,924,neutrofiilid,78,%,40.0,80.0
94146,85772,940,966,neutrofiilid,8,%,0.0,5.0
94147,85772,132,139,CRP,330,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
98809,85772,293,317,MCHC,317,g/L,317.0,357.0
98810,85772,321,340,PLT,523,NaN,145.0,390.0
98811,85772,201,222,WBC,12,NaN,3.5,8.8
98812,85772,231,251,RBC,2.95,NaN,3.9,5.2


In [103]:
pd.set_option('display.max_colwidth', 1)


In [106]:
 DS[85772:85773]

,text_ID,start,end,object,value,unit,min,max,text
85772,230794,96,111,Pikkus,173.0,cm,NaN,NaN,"18.11.2014 - õde <ANONYM id=""0"" type=""per"" morph=""_H_ sg n;_H_ sg n""/>\n\nRR: 110/60 mmHg\nKaal: 68,5 kg\nPikkus: 173 cm\nVÜM: 86 cm\nKMI: 23\nSuitsetamine: ei\n18.11.2014 - REFRAKTSIOON sph cyl ax\n OD -2,25 -0,25 18\n OS -2,0 -0,5 39\n\nVISUS KAUGELE \noma prilliga \nOD cc -1,25 -0,5 90 1,0\nOS cc -1,25 -0,5 60 1,0\n\nMõlemad silmad : cc 1,0\n\nVISUS LÄHEDALE \nKORREKTSIOONITA / - GA\nOD \nOS \nMõlemad silmad: 0,8\n\n18.11.2014 - Üldseisund hea.\n Nahk ja lim.kestad puhtad. \n Visuaalselt kilpn suurenenud\nSüdame tegevus regulaarne, toonid puhtad.\nKopsudes bilater vesikul. hingamiskahin\nLuu-lihassüst-patol.leiuta. Laseque bilater 90° \nLülisamba liikuvus nimmest N, õla- küünar- ja randmeliigeste liikuvus N, liigutused valuvabad.\nPasternatski smptm.bilater.negat.\nParavertebraalsed np-d palpatoorselt valutud C ja Th.pk-s\nVeenilaiendeid ei esine\n Tunnelsündroomid (Tinel, Phalen) bilater negat.\nKPR d=s\nEKG: regul SR fr.68 x min, N de ja repolarisatsiooni voltaaz,semiverik el posits. ES-, N EKG.\n\n\n."


In [107]:
print( DS[85772:85773]["text"])

85772    18.11.2014 -  õde <ANONYM id="0" type="per" morph="_H_ sg n;_H_ sg n"/>\n\nRR:  110/60  mmHg\nKaal:  68,5  kg\nPikkus: 173  cm\nVÜM: 86 cm\nKMI:  23\nSuitsetamine: ei\n18.11.2014 -  REFRAKTSIOON    sph    cyl    ax\n                        OD -2,25 -0,25 18\n                        OS -2,0 -0,5 39\n\nVISUS KAUGELE                                      \noma prilliga                            \nOD    cc -1,25 -0,5 90 1,0\nOS  cc -1,25 -0,5 60 1,0\n\nMõlemad silmad :   cc 1,0\n\nVISUS LÄHEDALE  \nKORREKTSIOONITA  / - GA\nOD                                 \nOS               \nMõlemad silmad: 0,8\n\n18.11.2014 -  Üldseisund hea.\n Nahk ja lim.kestad puhtad. \n Visuaalselt kilpn suurenenud\nSüdame tegevus regulaarne, toonid puhtad.\nKopsudes bilater vesikul. hingamiskahin\nLuu-lihassüst-patol.leiuta. Laseque bilater 90° \nLülisamba liikuvus nimmest N, õla- küünar- ja randmeliigeste liikuvus N, liigutused valuvabad.\nPasternatski smptm.bilater.negat.\nParavertebraalsed np-d palpato

In [ ]:
class SkillsExtractorNN:

    def __init__(self, word_features_dim, dense_features_dim):

        lstm_input_phrase = keras.layers.Input(shape=(None, word_features_dim))
        lstm_input_cont = keras.layers.Input(shape=(None, word_features_dim))
        dense_input = keras.layers.Input(shape=(dense_features_dim,))

        lstm_emb_phrase = keras.layers.LSTM(256)(lstm_input_phrase)
        lstm_emb_phrase = keras.layers.Dense(128, activation='relu')(lstm_emb_phrase)

        lstm_emb_cont = keras.layers.LSTM(256)(lstm_input_cont)
        lstm_emb_cont = keras.layers.Dense(128, activation='relu')(lstm_emb_cont)

        dense_emb = keras.layers.Dense(512, activation='relu')(dense_input)
        dense_emb = keras.layers.Dense(256, activation='relu')(dense_emb)

        x = keras.layers.concatenate([lstm_emb_phrase, lstm_emb_cont, dense_emb])
        x = keras.layers.Dense(128, activation='relu')(x)
        x = keras.layers.Dense(64, activation='relu')(x)
        x = keras.layers.Dense(32, activation='relu')(x)

        main_output = keras.layers.Dense(2, activation='softplus')(x)

        self.model = keras.models.Model(inputs=[lstm_input_phrase, lstm_input_cont, dense_input],
                                        outputs=main_output)

        optimizer = keras.optimizers.Adam(lr=0.0001)

        self.model.compile(optimizer=optimizer, loss='binary_crossentropy')

        
        
def fit(self, x_lstm_phrase, x_lstm_context, x_dense, y,
        val_split=0.25, patience=5, max_epochs=1000, batch_size=32):

    x_lstm_phrase_seq = keras.preprocessing.sequence.pad_sequences(x_lstm_phrase)
    x_lstm_context_seq = keras.preprocessing.sequence.pad_sequences(x_lstm_context)

    y_onehot = onehot_transform(y)

    self.model.fit([x_lstm_phrase_seq, x_lstm_context_seq, x_dense],
                   y_onehot,
                   batch_size=batch_size,
                   pochs=max_epochs,
                   validation_split=val_split,
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)])


def predict(self, x_lstm_phrase, x_lstm_context, x_dense):

    x_lstm_phrase_seq = keras.preprocessing.sequence.pad_sequences(x_lstm_phrase)
    x_lstm_context_seq = keras.preprocessing.sequence.pad_sequences(x_lstm_context)

    y = self.model.predict([x_lstm_phrase_seq, x_lstm_context_seq, x_dense])

    return y


def onehot_transform(y):

    onehot_y = []

    for numb in y:
        onehot_arr = np.zeros(2)
        onehot_arr[numb] = 1
        onehot_y.append(np.array(onehot_arr))

    return np.array(onehot_y)


